# Test cross validation scalability

Let's compare cross validation timings using 1 vs 4 cores on compressed genotype matrices as well as dense `Float64` matrices. 

In [1]:
using Distributed
addprocs(4)

@everywhere begin
    using Revise
    using MendelIHT
    using SnpArrays
    using Random
    using GLM
    using DelimitedFiles
    using Test
    using Distributions
    using LinearAlgebra
    using CSV
    using DataFrames
    using StatsBase
    BLAS.set_num_threads(1) # remember to set BLAS threads to 1 !!!
end

┌ Info: Precompiling MendelIHT [921c7187-1484-5754-b919-5d3ed9ac03c4]
└ @ Base loading.jl:1278


## Univariate response with SnpLinAlg

In [2]:
n = 1000  # number of samples
p = 10000 # number of SNPs
k = 10    # number of causal SNPs per trait
d = Normal
l = canonicallink(d())

# set random seed for reproducibility
Random.seed!(2021)

# simulate `.bed` file with no missing data
x = simulate_random_snparray(undef, n, p)
xla = SnpLinAlg{Float64}(x, model=ADDITIVE_MODEL, center=true, scale=true) 

# intercept is the only nongenetic covariate
z = ones(n)
intercept = 1.0

# simulate response y, true model b, and the correct non-0 positions of b
Y, true_b, correct_position = simulate_random_response(xla, k, d, l, Zu=z*intercept);

In [4]:
# 1 cores
Random.seed!(2020)
@time mses_new = cv_iht(Y, xla, z, d=d(), l=l, parallel=false);

Progress: 100%|█████████████████████████████████████████| Time: 0:00:05




Crossvalidation Results:
	k	MSE
	1	1218.5008240345846
	2	842.5557378754057
	3	634.0406117147634
	4	487.58369505980653
	5	391.3933763254454
	6	305.31664542373767
	7	267.91347992536646
	8	243.05711924230695
	9	243.47585351013382
	10	245.64651577890663
	11	250.63099201132914
	12	253.98774650724235
	13	254.7946957226353
	14	255.89622437798954
	15	263.60686957526195
	16	269.06640759615067
	17	271.05816592183567
	18	274.4334327761434
	19	279.3301188489498
	20	284.5831155777

Best k = 8

  5.367460 seconds (22.97 M allocations: 519.797 MiB, 1.28% gc time)


In [6]:
# 4 cores
Random.seed!(2020)
@time mses_new = cv_iht(Y, xla, z, d=d(), l=l, parallel=true);

Progress: 100%|█████████████████████████████████████████| Time: 0:00:01




Crossvalidation Results:
	k	MSE
	1	1218.5008240345846
	2	842.5557378754057
	3	634.0406117147634
	4	487.58369505980653
	5	391.3933763254454
	6	305.31664542373767
	7	267.91347992536646
	8	243.05711924230695
	9	243.47585351013382
	10	245.64651577890663
	11	250.63099201132914
	12	253.98774650724235
	13	254.7946957226353
	14	255.89622437798954
	15	263.60686957526195
	16	269.06640759615067
	17	271.05816592183567
	18	274.4334327761434
	19	279.3301188489498
	20	284.5831155777

Best k = 8

  1.833720 seconds (29.07 k allocations: 6.701 MiB, 0.52% gc time)


## Univariate response with dense Float64s

In [16]:
n = 5000  # number of samples
p = 10000 # number of SNPs
k = 10    # number of causal SNPs per trait
d = Normal
l = canonicallink(d())

# set random seed for reproducibility
Random.seed!(2021)

# simulate `.bed` file with no missing data
x = randn(n, p)

# intercept is the only nongenetic covariate
z = ones(n)
intercept = 1.0

# simulate response y, true model b, and the correct non-0 positions of b
y, true_b, correct_position = simulate_random_response(x, k, d, l, Zu=z*intercept);

In [17]:
# 1 cores
Random.seed!(2020)
@time mses_new = cv_iht(y, x, z, d=d(), l=l, parallel=false);

Cross validating...100%|████████████████████████████████| Time: 0:00:18




Crossvalidation Results:
	k	MSE
	1	7620.678703424767
	2	4843.2090739422965
	3	3823.4696319526106
	4	2882.8736571364025
	5	2057.0297220866105
	6	1711.3860665095106
	7	1296.1935231845227
	8	1112.6263164938034
	9	984.7885109603233
	10	967.4921113496174
	11	966.0060438329298
	12	968.2783508468474
	13	971.361243918521
	14	973.4022898469375
	15	978.1775331064498
	16	983.9153752806435
	17	983.0212506386123
	18	986.9111942083202
	19	988.6770001959334
	20	991.9368900330937

Best k = 11

 18.232142 seconds (74.80 M allocations: 1.347 GiB, 2.04% gc time)


In [18]:
# 4 cores
Random.seed!(2020)
@time mses_new = cv_iht(y, x, z, d=d(), l=l, parallel=true);

Cross validating...100%|████████████████████████████████| Time: 0:00:22




Crossvalidation Results:
	k	MSE
	1	7620.678703424767
	2	4843.2090739422965
	3	3823.4696319526106
	4	2882.8736571364025
	5	2057.0297220866105
	6	1711.3860665095106
	7	1296.1935231845227
	8	1112.6263164938034
	9	984.7885109603233
	10	967.4921113496174
	11	966.0060438329298
	12	968.2783508468474
	13	971.361243918521
	14	973.4022898469375
	15	978.1775331064498
	16	983.9153752806435
	17	983.0212506386123
	18	986.9111942083202
	19	988.6770001959334
	20	991.9368900330937

Best k = 11

 22.953429 seconds (47.78 k allocations: 10.283 MiB)


## Multivariate response with SnpLinAlg

In [22]:
n = 1000  # number of samples
p = 10000 # number of SNPs
k = 10    # number of causal SNPs per trait
r = 2

# set random seed for reproducibility
Random.seed!(2021)

# simulate `.bed` file with no missing data
x = simulate_random_snparray(undef, n, p)
xla = SnpLinAlg{Float64}(x, model=ADDITIVE_MODEL, center=true, scale=true) 

# intercept is the only nongenetic covariate
z = ones(n)
intercept = 1.0

# simulate response y, true model b, and the correct non-0 positions of b
Y, true_Σ, true_b, correct_position = simulate_random_response(xla, k, r, Zu=z*intercepts, overlap=2);

In [23]:
# 1 core
Random.seed!(2020)
Yt = Matrix(Y')
Zt = Matrix(z')
@time mses = cv_iht(Yt, Transpose(xla), Zt, path=1:20, parallel=false);

Cross validating...100%|████████████████████████████████| Time: 0:03:18




Crossvalidation Results:
	k	MSE
	1	2888.7160633632484
	2	2560.135862053543
	3	2067.943067029389
	4	1812.0395079444284
	5	1554.3120367449362
	6	1277.3237598020087
	7	1154.9320629872832
	8	1098.5910963871872
	9	1019.4597637296985
	10	1030.1412464715597
	11	1023.5545874904792
	12	1007.9022110997687
	13	1012.6193656356761
	14	1019.1491606608182
	15	1024.6877890077092
	16	1022.9300595671257
	17	1040.0286509856787
	18	1033.345570850089
	19	1039.8828186471897
	20	1036.274158344765

Best k = 12

198.895800 seconds (8.46 M allocations: 780.823 MiB, 0.07% gc time)


In [24]:
# 1 core
Random.seed!(2020)
Yt = Matrix(Y')
Zt = Matrix(z')
@time mses = cv_iht(Yt, Transpose(xla), Zt, path=1:20, parallel=true);

┌ Warning: Replacing docs for `MendelIHT.cv_iht :: Union{Tuple{T}, Tuple{Union{AbstractArray{T,1}, AbstractArray{T,2}},AbstractArray{T,2},Union{AbstractArray{T,1}, AbstractArray{T,2}}}} where T<:Union{Float32, Float64}` in module `MendelIHT`
└ @ Base.Docs docs/Docs.jl:227
┌ Warning: Replacing docs for `MendelIHT.cv_iht :: Union{Tuple{T}, Tuple{Union{AbstractArray{T,1}, AbstractArray{T,2}},AbstractArray{T,2},Union{AbstractArray{T,1}, AbstractArray{T,2}}}} where T<:Union{Float32, Float64}` in module `MendelIHT`
└ @ Base.Docs docs/Docs.jl:227
┌ Warning: Replacing docs for `MendelIHT.cv_iht :: Union{Tuple{T}, Tuple{Union{AbstractArray{T,1}, AbstractArray{T,2}},AbstractArray{T,2},Union{AbstractArray{T,1}, AbstractArray{T,2}}}} where T<:Union{Float32, Float64}` in module `MendelIHT`
└ @ Base.Docs docs/Docs.jl:227
┌ Warning: Replacing docs for `MendelIHT.cv_iht :: Union{Tuple{T}, Tuple{Union{AbstractArray{T,1}, AbstractArray{T,2}},AbstractArray{T,2},Union{AbstractArray{T,1}, AbstractArray{T,2



Crossvalidation Results:
	k	MSE
	1	2888.7160633632484
	2	2560.135862053543
	3	2067.943067029389
	4	1812.0395079444284
	5	1554.3120367449362
	6	1277.3237598020087
	7	1154.9320629872832
	8	1098.5910963871872
	9	1019.4597637296985
	10	1030.1412464715597
	11	1023.5545874904792
	12	1007.9022110997687
	13	1012.6193656356761
	14	1019.1491606608182
	15	1024.6877890077092
	16	1022.9300595671257
	17	1040.0286509856787
	18	1033.345570850089
	19	1039.8828186471897
	20	1036.274158344765

Best k = 12

 62.852016 seconds (1.03 M allocations: 57.195 MiB, 0.03% gc time)


## Multivariate response with dense Float64s

In [7]:
n = 1000  # number of samples
p = 10000 # number of SNPs
k = 10    # number of causal SNPs
r = 2     # number of traits

# set random seed for reproducibility
Random.seed!(2021)

# simulate `.bed` file with no missing data
x = randn(n, p)

# intercept is the only nongenetic covariate
z = ones(n, 1)
intercepts = [10.0 1.0] # each trait have different intercept

# simulate response y, true model b, and the correct non-0 positions of b
Y, true_Σ, true_b, correct_position = simulate_random_response(x, k, r, Zu=z*intercepts, overlap=2);

In [8]:
# 1 core
Random.seed!(2020)
Yt = Matrix(Y')
Zt = Matrix(z')
@time mses = cv_iht(Yt, Transpose(x), Zt, path=1:20, parallel=false);

Cross validating...100%|████████████████████████████████| Time: 0:02:03




Crossvalidation Results:
	k	MSE
	1	2629.5253388334654
	2	2445.3327091797446
	3	1699.7479496810877
	4	1581.049584753384
	5	1345.7979096211036
	6	1002.831521953042
	7	938.1884998991206
	8	731.4755412240933
	9	723.5331961068099
	10	826.2287168502997
	11	605.2207412646168
	12	608.5209443463392
	13	609.4258432524741
	14	608.3389786061807
	15	607.6231927917379
	16	610.1537812223704
	17	610.4358299991871
	18	612.6346959329958
	19	609.484474645297
	20	611.8138514213124

Best k = 11

123.398046 seconds (8.41 M allocations: 1.157 GiB, 0.16% gc time)


In [9]:
# 4 core
Random.seed!(2020)
Yt = Matrix(Y')
Zt = Matrix(z')
@time mses = cv_iht(Yt, Transpose(x), Zt, path=1:20, parallel=true);

Cross validating...100%|████████████████████████████████| Time: 0:00:41




Crossvalidation Results:
	k	MSE
	1	2629.5253388334654
	2	2445.3327091797446
	3	1699.7479496810877
	4	1581.049584753384
	5	1345.7979096211036
	6	1002.831521953042
	7	938.1884998991206
	8	731.4755412240933
	9	723.5331961068099
	10	826.2287168502997
	11	605.2207412646168
	12	608.5209443463392
	13	609.4258432524741
	14	608.3389786061807
	15	607.6231927917379
	16	610.1537812223704
	17	610.4358299991871
	18	612.6346959329958
	19	609.484474645297
	20	611.8138514213124

Best k = 11

 41.739760 seconds (463.52 k allocations: 26.606 MiB)
